<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Centre un titre</title>
    <style>
        h1 {
            text-align: center;
        }
    </style>
</head>

<h1 align:center>  Breast Cancer: Causes and Drug Target Prediction </h1>

The program analyzes the cause of breast cancer from a Boolean network datamodel, namely which genes should be mutated to cause cancer and are they oncogenes (True) or tumor suppressors (False).
Next, the program found the potential therapeutic targets and the actions on them for a cancer caused by BRCA1 mutation.

In [ ]:
import time
import matplotlib.pyplot as plt
from sympy import SOPform, simplify_logic
from tabulate import tabulate
from pulp import PULP_CBC_CMD
from boon import *

## Network model
import the network model from a text file. Note that this model also exists in BooN format.
Note that in a normal cell, these two phenotypes exist potentially.
The model is a Boolean network connecting genes involved in Beast cancer.

In [ ]:
#  Load the breast cancer network model (bcbn). A version of the model in BNet is also available 
bcbn = BooN.load("breast-cancer")

# Show the Boolean network
print(bcbn)

### List of genes

In [ ]:
# print all the genes.
print(bcbn.variables)

### Stable States
In this model, two stable states exist that respectively correspond to cell division and apoptosis, identified by the Boolean version of the biomarker states.

In [ ]:
stable_states = bcbn.stable_states
print(tabulate(stable_states, headers='keys', tablefmt='grid'))

### Interaction Graph

In [ ]:
bcbn.draw_IG()
plt.show()  # Show the interaction graph

## Cause of Cancer
Firstly, to figure out the potential causes of cancer, a controllability analysis is used.
This analysis is based on the characterization of biomarkers, which are genes whose states are used to differentiate between phenotypes.
These biomarkers are CycD1 and Bax
Based on the states of the aforementioned biomarkers. A control query is then defined with the goal of identifying the cause of cancer.

### Characterization of the biomarkers
The selected biomarkers are CycD1 and Bax. The Boolean profiles, respectively, correspond to cell division and apoptosis.


| CyCD1 | Bax |     *Phenotype* |
|:-----:|:---:|----------------:|
|   0   |  1  |     *Apoptosis* |
|   1   |  0  | *Cell Division* |


In [ ]:
CycD1, Bax = symbols("CycD1 Bax")
biomarkers = {CycD1, Bax}

In essence, the model represents the cancerous state as a loss of apoptosis, which is a hallmark of cancer.
We will define a query or formula. That delineates the circumstances under which apoptosis cannot be sustained in any stable state.

In [ ]:
# Characterize a marking corresponding to the apoptosis
marking = {CycD1: False, Bax: True}
# convert the marking into a formula
apoptosis_marking = SOPform(biomarkers, [marking])
print("Apoptosis biomarkers")
print(tabulate([marking], headers='keys', tablefmt='plain'))


### Control query definition
Set the marking formula of cancer losing the apoptosis, that is the negation of apoptosis query, A cell therefore cannot die defining a hallmark of cancer.

In [ ]:
kquery = simplify_logic(~apoptosis_marking, force=True)
print("Disease query, prevent apoptosis: ", kquery)

Now, we define the variables where the control can be applied to force the freeze of variables.
A variable can have two controls for freezing to True and to False.
The biomarkers are always excluded from the control since they are observers characterizing the phenotypes by their profile.

In [ ]:
frozenvars0 = frozenvars1 = bcbn.variables - biomarkers
print("Frozen variables 0 and 1")
print(tabulate([frozenvars0, frozenvars1], tablefmt='grid'))

Finally, we extend the Boolean network to a Boolean Controlled network according to the frozen variables. 

### Boolean Controlled Network (BCN)
Copy the network and fix the control on it. The control consists in adding control parameters such that: 
- _u0X: is the control for setting the variable X to False
- _u1X: is the control for setting the variable X to True.

A control is active when it freezes (set to 0/False). 


In [ ]:
bcc = bcbn.copy()
bcc.control(frozenvars0, frozenvars1)
print(bcc)

 ### Controllability Analysis
A controllability analysis should be conducted for identifying the potential gene freezes that would satisfy the query at stable state.

The number of models is approximately 3,500. We kindly request your patience.

In [ ]:
print("\nQuery: Necessary avoid the apoptosis.")
start_time = time.time()
 # The trace is activated to see the evolution of the computation.
destiny = bcc.necessary(kquery, trace=True) 
print("Runtime..: {:5.2f}".format(time.time() - start_time))
print("# clauses: {}".format(len(destiny.args)))
print("# symbols: {}".format(destiny.count(Symbol)))

### Controllability Analysis
About 10 solutions

In [ ]:
print("\nCore")
# Compute the core.
start_time = time.time()
# The standard Pulp solver (Default) is used. However, other solvers as GUROBI can be used for improving the performance of the resolution.
core = bcc.destify(destiny, trace=True, solver=PULP_CBC_CMD)
print("Runtime: {:5.2f}".format(time.time() - start_time))

From the core, we derive the potential freeze actions as a pair (gene, freeze action).
The table comprises a series of lines, each corresponding to a specific freeze action on genes that is necessary to fulfil the query.
In the event that two pairs are present within a single line, it is necessary to combine both actions.

In [ ]:
print("\nActions")  # Transform the core into actions.
actions = core2actions(core)
print(tabulate(actions))

### Define the possibility
It is important to note that the calculation of the possibility itself results in an empty set of controls,
since an equilibrium already exists which does not correspond to the apoptosis profile.
Consequently, no control is required.

In [ ]:
destiny2 = bcc.possibly(kquery)
print("# clauses: {}".format(len(destiny2.args)))

In [ ]:
core2 = bcc.destify(destiny2)
print("\nActions")
actions2 = core2actions(core2)
print(tabulate(actions2))

## Drug Target identification
We now identify the possible drug target associated with the expected action to cure BRCA1 Cancer.

In [ ]:
# We define a situation of cancer with BRCA1 mutation.
print("\n" * 2, " - Mutate BRCA1  -")
BRCA1 = symbols("BRCA1")
bcbn.desc[BRCA1] = False

print("Mutated network.")
print(bcbn)

Define the frozen variables from the previous situation of the frozen variables by removing BRCA1

In [ ]:
# copy the network and apply control
# Remove the mutated gene from the controlled variable since no modifications is possible.
frozenvars0 = frozenvars1 = frozenvars0 - {BRCA1}
bndrug = bcbn.copy()  # fix control on a new instance of bcbn.
bndrug.control(frozenvars0, frozenvars1)

### Controllability analysis
The issue is to identify the actions that induce apoptosis to determine the most effective therapeutic targets.
It is hypothesized that at least a stable state will fulfil the query property since the drug may not work for all people.

Note that the possibility of resolution is faster than the necessity.

In [ ]:

print("\nQuery : Possibly lead to apoptosis.")
start_time = time.time()
destiny = bndrug.possibly(apoptosis_marking)
print("Runtime..: {:5.2f}".format(time.time() - start_time))
print("# clauses: {}".format(len(destiny.args)))
print("# symbols: {}".format(destiny.count(Symbol)))

In [ ]:
print("\nCore")
start_time = time.time()
core = bndrug.destify(destiny)
print("Runtime: {:5.2f}".format(time.time() - start_time))

From the core, we finally derive the potential freeze actions as a pair (gene, freeze action) to gain the apoptosis.

In [ ]:
print("\nActions")  # Transform the core into actions.
actions = core2actions(core)
print(tabulate(actions))

#### Switch to necessary condition
From the solutions we filter those where the query holds to all stable states corresponding to a necessary condition.
Notice that the method must be applied to the original non-controlled network.

In [ ]:
core_necessary = bcbn.filter_necessary(destiny,core,trace=False)
actions = core2actions(core_necessary)
print("\nNecessary Actions")
print(tabulate(actions))